In [1]:
# Setup notebook
import random
import numpy as np
import evaluation.plot as plot

# Load the autoreload extension
%load_ext autoreload

# Set autoreload to reload all modules every time a cell is executed
%autoreload 2

seed = 0
random.seed(seed)
np.random.seed(seed)

In [2]:
from dataset.common import resolve_folds
from dataset.movie_lens import load_ml_100k_folds

data, k_fold = load_ml_100k_folds()
folds = resolve_folds(data, k_fold)

folds_without_index = [fold[1] for fold in folds]

Already downloaded!. Nothing to do.


In [3]:
from evaluation.strategies import (
    MAEStrategy,
    RMSEStrategy,
    MicroAveragedRecallStrategy,
    MacroAveragedRecallStrategy,
    RecallAtKStrategy,
    MicroAveragedPrecisionStrategy,
    MacroAveragedPrecisionStrategy,
    PrecisionAtKStrategy,
    NDCGStrategy,
    F1ScoreStrategy,
    PredictionCoverageStrategy,
    BiclusterCountStrategy,
    MeanBiclusterSizeStrategy,
    MeanBiclusterIntentStrategy,
    MeanBiclusterExtentStrategy,
    ItemCoverage,
    UserCoverage,
)

train_measures = [
    BiclusterCountStrategy(),
    MeanBiclusterSizeStrategy(),
    MeanBiclusterIntentStrategy(),
    MeanBiclusterExtentStrategy(),
    # ItemCoverage(),
    # UserCoverage(),
]

test_measures = [
    MAEStrategy(verbose=False),
    RMSEStrategy(verbose=False),
    MicroAveragedRecallStrategy(threshold=4.0),
    MacroAveragedRecallStrategy(threshold=4.0),
    RecallAtKStrategy(k=20, threshold=4.0),
    MicroAveragedPrecisionStrategy(threshold=4.0),
    MacroAveragedPrecisionStrategy(threshold=4.0),
    PrecisionAtKStrategy(k=20, threshold=4.0),
    NDCGStrategy(k=20, threshold=4.0),
    F1ScoreStrategy(k=20, threshold=4.0),
    PredictionCoverageStrategy(),
]

In [4]:
from pattern_mining.strategies import QUBIC2Strategy
from recommenders.PAkNN import PAkNN
from recommenders.common import weight_frequency
from evaluation.benchmark import GridSearch, generate_parameter_combinations


qubic_parameters_grid = {
    "bicluster_number": [100000],
    "max_overlap": [0.35, 0.75, 1.0],
    "consistency": [0.75, 1.0],
    "minimum_column_width": [5, 20, 50, 100],
}

qubic_strategies = [
    QUBIC2Strategy(**named_parameters)
    for named_parameters in generate_parameter_combinations(qubic_parameters_grid)
]

parameters_grid = {
    "mining_strategy": qubic_strategies,
    "knn_type": ["item"],
    "number_of_top_k_biclusters": [10, 50, 100, 200],
    "knn_k": [20, 50, 100, 200],
}

grid_search = GridSearch(
    PAkNN,
    parameters_grid,
    test_measures,
    train_measures,
    max_workers=16,
)

from numba.core.errors import NumbaWarning
import warnings


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=NumbaWarning)
    best, ordering, raw = grid_search.fit(folds_without_index)

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar divide

/home/brodrigues/Projects/RecoBinaPs/recommenders/PAkNN.py:119: RuntimeWarning:

divide by zero encountered in scalar 

In [ ]:
import json
from evaluation.benchmark import FallbackEncoder

with open('IB_PAKNN_QUBIC_grid_best.results', 'w') as file_object:
    json.dump(best, file_object, indent=4, cls=FallbackEncoder)

with open('IB_PAKNN_QUBIC_grid_ordering.results', 'w') as file_object:
    json.dump(ordering, file_object, indent=4, cls=FallbackEncoder)

with open('IB_PAKNN_QUBIC_grid_raw.results', 'w') as file_object:
    json.dump(raw, file_object, indent=4, cls=FallbackEncoder)